In [34]:
import pandas as pd

## Load the datasets

In [35]:
# Load all relevant sheets into separate DataFrames
df_states = pd.read_excel("Giardia Cases Data.xlsx", sheet_name='States')
df_age = pd.read_excel("Giardia Cases Data.xlsx", sheet_name='Age')
df_sex = pd.read_excel("Giardia Cases Data.xlsx", sheet_name='Sex')


## Remove invalid values

In [36]:
# Define the values to remove
invalid_values = ["Not Applicable", "Not Reportable", "Not Available"]

# Remove rows where any cell contains one of the invalid values
df_states_cleaned = df_states[~df_states.isin(invalid_values).any(axis=1)]

## Creating separate rows for Males and Females

In [37]:
# Duplicate every row by concatenating the DataFrame with itself
df_states_duplicated = pd.concat([df_states_cleaned] * 2, ignore_index=True)

df_states_final = df_states_duplicated[~df_states_duplicated.apply(lambda row: row.astype(str).str.contains("United States", case=False)).any(axis=1)]

In [38]:
# Make sure column names are correct (e.g., 'year' and 'state')
df_states_sorted = df_states_final.sort_values(by=['Year', 'States'], ascending=[True, True]).reset_index(drop=True)

In [39]:
# Step 1: Drop duplicates by state and year to have one row per state-year
df_unique = df_states_sorted.drop_duplicates(subset=['States', 'Year'])

# Step 2: Create Sex DataFrame
sex_df = pd.DataFrame({"Sex": ["Male", "Female"]})

# Step 3: Cross join to get exactly 2 rows per state-year
df_expanded = df_unique.merge(sex_df, how='cross').reset_index(drop=True)

df_expanded

,Year,States,State_Case_Count,State_Population,State_Incidence,Sex
0,2016,Alabama,234,4863300,4.81,Male
1,2016,Alabama,234,4863300,4.81,Female
2,2016,Alaska,86,741894,11.59,Male
3,2016,Alaska,86,741894,11.59,Female
4,2016,Arizona,125,6931071,1.8,Male
...,...,...,...,...,...,...
613,2022,West Virginia,92,1775156,5.18,Female
614,2022,Wisconsin,418,5892539,7.09,Male
615,2022,Wisconsin,418,5892539,7.09,Female
616,2022,Wyoming,43,581381,7.4,Male


## Cleaning the Sex dataset

In [41]:
df_sex = df_sex.loc[:, ~df_sex.columns.str.startswith("Unnamed")]

In [42]:
df_sex = df_sex[(df_sex["Sex"] != "Unknown") & (df_sex["Year"] >= 2016)]


In [43]:
df_sex

,Sex,Number of Cases,Incidence,Year,Population by Sex
15,Male,9770.0,7.82,2016,124892519.0
16,Female,6490.0,5.04,2016,128799022.0
18,Male,9354.0,7.43,2017,125880430.0
19,Female,5792.0,4.46,2017,129749916.0
21,Male,9561.0,61.40,2018,126342343.0
22,Female,5966.0,38.30,2018,130204226.0
24,Male,9314.0,62.60,2019,126683266.0
25,Female,5513.0,37.00,2019,130546515.0
27,Male,5661.0,4.46,2020,127058910.0
28,Female,3757.0,2.87,2020,130941100.0


## Combining the Incidence rates for both the sexes with our main dataset. Values for 2016 and 2017 were NaNs, so entering them manually.

In [10]:
df_expanded = df_expanded.merge(
    df_sex[["Year", "Sex", "Incidence"]],  # from df_sex
    how="left",
    left_on=["Year", "Sex"],               # match df_states
    right_on=["Year", "Sex"]
)

In [11]:
df_expanded.rename(columns={"Incidence": "Sex_Incidence"}, inplace=True)

In [12]:
df_expanded.loc[(df_expanded["Year"] == 2016) & (df_expanded["Sex"] == "Male") & (df_expanded["Sex_Incidence"].isna()), "Sex_Incidence"] = 7.82

df_expanded.loc[(df_expanded["Year"] == 2017) & (df_expanded["Sex"] == "Male") & (df_expanded["Sex_Incidence"].isna()), "Sex_Incidence"] = 7.43

In [13]:
df_expanded

,Year,States,State_Case_Count,State_Population,State_Incidence,Sex,Sex_Incidence
0,2016,Alabama,234,4863300,4.81,Male,7.82
1,2016,Alabama,234,4863300,4.81,Female,5.04
2,2016,Alaska,86,741894,11.59,Male,7.82
3,2016,Alaska,86,741894,11.59,Female,5.04
4,2016,Arizona,125,6931071,1.8,Male,7.82
...,...,...,...,...,...,...,...
613,2022,West Virginia,92,1775156,5.18,Female,38.10
614,2022,Wisconsin,418,5892539,7.09,Male,61.50
615,2022,Wisconsin,418,5892539,7.09,Female,38.10
616,2022,Wyoming,43,581381,7.4,Male,61.50


# Combining the population for each sex with the main dataset

The idea is to calculate the proportion of the males and females being tested, multiply the proportions with the population of each state being tested, and using the incidence rates to calculate the number of cases for each state categorised by sex.

In [14]:
df_sex["Sex_Proportion"] = (
    df_sex["Population by Sex"] /
    df_sex.groupby("Year")["Population by Sex"].transform("sum")
)

### Calculating proportion of each sex as per the population

In [15]:
df_proportions = (
    df_sex.groupby(["Year", "Sex"], as_index=False)["Population by Sex"]
    .sum()
)

df_proportions["Sex_Proportion"] = (
    df_proportions["Population by Sex"] /
    df_proportions.groupby("Year")["Population by Sex"].transform("sum")
)

In [16]:
df_expanded = df_expanded.merge(
    df_proportions[["Year", "Sex", "Sex_Proportion"]],
    how="left",
    on=["Year", "Sex"]
)

In [17]:
df_expanded.loc[(df_expanded["Year"] == 2016) & (df_expanded["Sex"] == "Male") & (df_expanded["Sex_Proportion"].isna()), "Sex_Proportion"] = 0.492301

df_expanded.loc[(df_expanded["Year"] == 2017) & (df_expanded["Sex"] == "Male") & (df_expanded["Sex_Proportion"].isna()), "Sex_Proportion"] = 0.492431

### Multiplying the proportions with the state population to get the number of males and females in each state that are being tested

In [18]:
df_expanded["State_Sex_Population"] = (
    df_expanded["Sex_Proportion"] * df_expanded["State_Population"]
)


In [19]:
df_expanded

,Year,States,State_Case_Count,State_Population,State_Incidence,Sex,Sex_Incidence,Sex_Proportion,State_Sex_Population
0,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533
1,2016,Alabama,234,4863300,4.81,Female,5.04,0.507699,2469094.086557
2,2016,Alaska,86,741894,11.59,Male,7.82,0.492301,365235.158094
3,2016,Alaska,86,741894,11.59,Female,5.04,0.507699,376659.07681
4,2016,Arizona,125,6931071,1.8,Male,7.82,0.492301,3412173.184371
...,...,...,...,...,...,...,...,...,...
613,2022,West Virginia,92,1775156,5.18,Female,38.10,0.504076,894813.682363
614,2022,Wisconsin,418,5892539,7.09,Male,61.50,0.495924,2922251.024713
615,2022,Wisconsin,418,5892539,7.09,Female,38.10,0.504076,2970287.975287
616,2022,Wyoming,43,581381,7.4,Male,61.50,0.495924,288320.743061


### Calculating the number of cases using the incidence rates for each state. 

For example, the population of females being tested in Alabama is 400,000, and the incidence rate per 100,000 for Alabama is 5. So, the number of cases for females in Alabama would be 400,000 * 5/100,000 = 20.

In [20]:
df_expanded["State_Sex_CaseCount"] = (
    df_expanded["State_Sex_Population"] * df_expanded["State_Incidence"] / 100000
)


In [21]:
df_expanded["State_Sex_CaseCount"] = pd.to_numeric(
    df_expanded["State_Sex_CaseCount"], errors="coerce"
)

df_expanded["State_Sex_CaseCount"] = (
    df_expanded["State_Sex_CaseCount"].round().astype("Int64")
)

In [22]:
df_expanded

,Year,States,State_Case_Count,State_Population,State_Incidence,Sex,Sex_Incidence,Sex_Proportion,State_Sex_Population,State_Sex_CaseCount
0,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115
1,2016,Alabama,234,4863300,4.81,Female,5.04,0.507699,2469094.086557,119
2,2016,Alaska,86,741894,11.59,Male,7.82,0.492301,365235.158094,42
3,2016,Alaska,86,741894,11.59,Female,5.04,0.507699,376659.07681,44
4,2016,Arizona,125,6931071,1.8,Male,7.82,0.492301,3412173.184371,61
...,...,...,...,...,...,...,...,...,...,...
613,2022,West Virginia,92,1775156,5.18,Female,38.10,0.504076,894813.682363,46
614,2022,Wisconsin,418,5892539,7.09,Male,61.50,0.495924,2922251.024713,207
615,2022,Wisconsin,418,5892539,7.09,Female,38.10,0.504076,2970287.975287,211
616,2022,Wyoming,43,581381,7.4,Male,61.50,0.495924,288320.743061,21


# Introducing Age as another variables

We have 7 age categories.

In [23]:
# Define the categories
age_categories = [
    "<1 year", 
    "1-4 years", 
    "5-14 years", 
    "15-24 years", 
    "25-39 years", 
    "40-64 years", 
    "65+ years"
]

# Expand the dataframe so every row is duplicated for each Age_Category
df_ages = (
    df_expanded
    .assign(key=1)
    .merge(pd.DataFrame({"Age_Category": age_categories, "key": [1]*len(age_categories)}), on="key")
    .drop("key", axis=1)
)

In [24]:
df_ages

,Year,States,State_Case_Count,State_Population,State_Incidence,Sex,Sex_Incidence,Sex_Proportion,State_Sex_Population,State_Sex_CaseCount,Age_Category
0,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,<1 year
1,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,1-4 years
2,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,5-14 years
3,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,15-24 years
4,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,25-39 years
...,...,...,...,...,...,...,...,...,...,...,...
4321,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,5-14 years
4322,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,15-24 years
4323,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,25-39 years
4324,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,40-64 years


## Preparing the dataset for calculations

We will use the similar process to calculate the number of cases for each state, categorised by age and sex of the person.

In [25]:
# Merge on both Year and Age_Category
df_ages = df_ages.merge(
    df_age[["Year", "Age_Category", "Age_Population", "Age_Incidence"]],
    on=["Year", "Age_Category"],
    how="left"
)

In [26]:
df_ages["Age_Proportion"] = (
    df_ages["Age_Population"] /
    df_ages.groupby("Year")["Age_Population"].transform("sum")
)

In [27]:
# Collapse by Year + Sex + Age_Category
age_totals = df_ages.groupby(["Year", "Sex", "Age_Category"], as_index=False)["Age_Population"].sum()

# Compute proportions within each Year + Sex
age_totals["Age_Proportion"] = (
    age_totals.groupby(["Year", "Sex"])["Age_Population"].transform(lambda x: x / x.sum())
)


In [28]:
df_ages = df_ages.merge(
    age_totals[["Year", "Sex", "Age_Category", "Age_Proportion"]],
    on=["Year", "Sex", "Age_Category"],
    how="left"
)


In [29]:
df_ages["State_Sex_Age_Population"] = (
    df_ages["Age_Proportion_y"] * df_ages["State_Sex_Population"]
)


In [30]:
df_ages["State_Sex_Age_CaseCount"] = (
    df_ages["State_Sex_Age_Population"] * df_ages["State_Incidence"] / 100000
)


In [32]:
df_ages["State_Sex_Age_Incidence"] = (
    df_ages["State_Sex_Age_CaseCount"] / df_ages["State_Sex_Age_Population"] * 100000
)


# The final dataset

In [33]:
df_ages

,Year,States,State_Case_Count,State_Population,State_Incidence,Sex,Sex_Incidence,Sex_Proportion,State_Sex_Population,State_Sex_CaseCount,Age_Category,Age_Population,Age_Incidence,Age_Proportion_x,Age_Proportion_y,State_Sex_Age_Population,State_Sex_Age_CaseCount,State_Sex_Age_Incidence
0,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,<1 year,3058322,3.40,0.000137,0.012055,28862.835939,1.388302,4.81
1,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,1-4 years,12301119,13.06,0.000551,0.048488,116091.49709,5.584001,4.81
2,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,5-14 years,31642189,6.04,0.001417,0.124727,298622.352341,14.363735,4.81
3,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,15-24 years,33918142,5.54,0.001519,0.133698,320101.60078,15.396887,4.81
4,2016,Alabama,234,4863300,4.81,Male,7.82,0.492301,2394207.4533,115,25-39 years,51193436,6.76,0.002293,0.201794,483136.747674,23.238878,4.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4321,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,5-14 years,31996119,4.53,0.001370,0.120595,35341.61048,2.615279,7.4
4322,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,15-24 years,34876842,3.72,0.001494,0.131453,38523.539831,2.850742,7.4
4323,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,25-39 years,53713645,6.06,0.002301,0.202450,59329.905575,4.390413,7.4
4324,2022,Wyoming,43,581381,7.4,Female,38.10,0.504076,293060.256939,22,40-64 years,83010687,5.43,0.003555,0.312872,91690.22548,6.785077,7.4


In [112]:
df_ages.to_csv("df_ages.csv", index=False)